In [1]:
! wget https://hktn2022.blob.core.windows.net/dataset/hist_data.csv
! wget https://hktn2022.blob.core.windows.net/dataset/test.csv

--2022-05-25 10:58:15--  https://hktn2022.blob.core.windows.net/dataset/hist_data.csv
Resolving hktn2022.blob.core.windows.net (hktn2022.blob.core.windows.net)... failed: Name or service not known.
wget: unable to resolve host address ‘hktn2022.blob.core.windows.net’
--2022-05-25 10:58:16--  https://hktn2022.blob.core.windows.net/dataset/test.csv
Resolving hktn2022.blob.core.windows.net (hktn2022.blob.core.windows.net)... failed: Name or service not known.
wget: unable to resolve host address ‘hktn2022.blob.core.windows.net’


In [ ]:
# Пример решения с использованием статистического подхода - подсчет 
# совстречаемостей.

import pandas as pd
import numpy as np
import gc

from collections import Counter 

hist_data = pd.read_csv('hist_data.csv')

# соберем словарь встречаемостей - какие item_id покупались чаще с 
# каждым item_id 
tmp = (
    hist_data[['item_id', 'pav_order_id']]
    .sort_values(['item_id', 'pav_order_id'])
    .merge(hist_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
)
tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))

most_freq_dict = {k: v for (k, v) in tmp1.iteritems()}

del tmp1, tmp
gc.collect()

test = pd.read_csv('test.csv')

In [ ]:
grouped = hist_data.groupby(by = 'buyer_id')['pav_order_id'].value_counts()
user_order_count = [0 for i in range(63925)]
it = 0
for x in hist_data['buyer_id'].unique():
  user_order_count[it] = (x, len(grouped[x]))
  it += 1
user_order_count = sorted(user_order_count, key = lambda x: x[1], reverse=True)

In [ ]:
from collections import defaultdict

In [ ]:
fucking_slaves = defaultdict(int)
for user in user_order_count:
  if user[1] > 2:
    fucking_slaves[user[0]] = 1

In [ ]:
from math import ceil
def get_rate(user_id):
  buys = hist_data[hist_data['buyer_id'] == user_id]['item_id'].value_counts()
  max_val = max(buys)
  min_val = min(buys)
  buys = buys*10/max_val
    
  return buys

In [ ]:
# из списка кандидатов по совстречаемости удаляем повторяющиеся item_id, 
# сохраняя порядок
def get_unique_recs(recs: list, top_n: int) -> list:
    rec_dict = {}
    counter = 0
    for k, v in recs:
        if k not in rec_dict:
            rec_dict[k] = v
            counter += 1
        if counter == top_n:
            break
    return list(rec_dict.keys())

def rec_by_item(item_id: int, most_freq_dict: dict) -> list:
    
    return most_freq_dict.get(item_id, None)

# для каждого item_id соберем top_n самых часто встречающихся item_id, 
# отсортируем по частоте и выберем уникальные
def rec_by_basket(buyer_id : int, basket: list, most_freq_dict: dict, top_n: int = 20) -> list:
    
    if buyer_id in fucking_slaves:
      most_rated = list(get_rate(buyer_id).index)
      reccomendation = []
      for item in most_rated:
        if item not in basket:
          reccomendation.append(item)
        if len(reccomendation) == top_n: #TODO: поменьять на top_n//2
          break
      if len(reccomendation) < top_n:
        print("ПИЗДА", len(reccomendation))
      return reccomendation

    res = []
    for item in basket:
        recs = rec_by_item(item, most_freq_dict)
        if recs is not None:
            res += recs
    
    res = sorted(res, key=lambda x: x[1], reverse=True)
    
    return get_unique_recs(res, top_n)

In [ ]:
from sklearn.model_selection import train_test_split

def split_data(data, test_size=0.3):
    orders_sort = data[['pav_order_id', 'created']].drop_duplicates().sort_values(by=['created', 'pav_order_id'])
    train_orders, test_orders = train_test_split(orders_sort['pav_order_id'].tolist(), test_size=test_size, shuffle=False)
    train_orders, test_orders = set(train_orders), set(test_orders)
    train = data[data['pav_order_id'].apply(lambda x: x in train_orders)]
    test = data[data['pav_order_id'].apply(lambda x: x in test_orders)]
    return train, test, orders_sort, train_orders, test_orders
    
def dcg(
    y_relevance: np.ndarray
) -> float:
    return np.sum([(2**i - 1) / np.log2(k + 1) for (k, i) in enumerate(y_relevance, start=1)])

def ndcg(
    y_relevance: np.ndarray,
    k: int
) -> float:
    if y_relevance.sum() == 0:
        return 0.0
    DCG = dcg(y_relevance[:k])
    IDCG = dcg(-np.sort(-y_relevance)[:k])
    return DCG / IDCG

def apply_relevance(x):
    return [int(item in x['basket']) for item in x['preds']]

def create_relevance(pred):
    d = pred.copy()
    d['basket'] = d['basket'].apply(set)
    d = d.apply(apply_relevance, axis=1)
    return d

def ndcg_full_dataset(d):
    dd = pd.DataFrame(d.to_list()).fillna(0).to_numpy()
    k = dd.shape[1]
    scores = [ndcg(dd[i], k) for i in range(len(dd))]
    return np.mean(scores)

def compute_ndcg_score(pred):
    relevance = create_relevance(pred)
    return ndcg_full_dataset(relevance)

def make_coocurs_dict(train_data):
    tmp = (
        train_data[['item_id', 'pav_order_id']]
        .sort_values(['item_id', 'pav_order_id'])
        .merge(train_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
    )
    tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
    tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))

    most_freq_dict = {k: v for (k, v) in tmp1.iteritems()}

    del tmp1, tmp
    gc.collect()
    return most_freq_dict

def create_basket(test_data):
    pred = test_data.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
    return pred

def make_predictions(test_data, most_freq_dict):
    pred = test.groupby(['pav_order_id', 'buyer_id'])['item_id'].agg([('basket', list)])
    pred.reset_index(inplace=True)
    xuy1 = []
    xuy2 = []
    for index, row in pred.iterrows():
      order_id = row['pav_order_id']
      preds = rec_by_basket(row['buyer_id'], row['basket'], most_freq_dict=most_freq_dict)
      xuy1.append(order_id)
      xuy2.append(preds)
    ans = pd.DataFrame({'pav_order_id': xuy1, 'preds': xuy2})
    return pred

In [ ]:
# считываем исторические данные
data = pd.read_csv("hist_data.csv", parse_dates=['created'])

# разобьем историю в отношении 70 на 30 для трейна и валидации
train_data, test_data, orders_sort, train_orders, test_orders = split_data(data)

# соберем словарь встречаемостей - какие item_id покупались чаще с каждым item_id 
most_freq_dict = make_coocurs_dict(train_data)
# предсказываем
pred = make_predictions(test_data, most_freq_dict)
pred.to_csv("preds_on_splitted_hist_data.csv")



In [ ]:
# посчитаем скор для всего набора предсказаний
d_score = compute_ndcg_score(pred)
print(d_score)